<a href="https://colab.research.google.com/github/jacquelinedoan/pricing_causal/blob/main/pricing_causal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Heckman Correction, Instrumental Variable, and DoubleML for Demand Modelling with Censored Purchase Data**

For a given customer $i$ and product $j$ (and context $x$), we want price $p^*$ such that
$$p^* = \text{argmax}_p \text{ profit} (p|x) = (p-c)\times E(D(p|x))$$

where
*   $c$: unit cost
*   $D(p|x)$: expected demand at price $p$ given context $x$

In [ ]:
!pip install doubleml

In [229]:
import os
import numpy as np
import pandas as pd
import kagglehub

pd.set_option('display.max_columns', None)

# Data

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
cust, sell, rev, item, prod, geo, cat, orders, pay = [pd.read_csv(f"{path}/{file}") for file in os.listdir(path)]
master = orders.merge(cust, on="customer_id", how='left')\
               .merge(item, on="order_id", how='left')\
               .merge(prod, on="product_id", how='left')\
               .merge(cat, on="product_category_name", how='left')\
               .merge(sell, on="seller_id", how='left')\
               .merge(pay, on="order_id", how='left')\
               .merge(rev, on="order_id", how='left')

# Drop entries where price is null
master.dropna(subset=['price'], inplace=True)
master.reset_index(drop=True, inplace=True)

# Handle Datetime data
master['order_purchase_timestamp'] = pd.to_datetime(master.order_purchase_timestamp,
                                             format='%Y-%m-%d %H:%M:%S')
master['order_estimated_delivery_date'] = pd.to_datetime(master.order_estimated_delivery_date,
                                              format='%Y-%m-%d %H:%M:%S')
master['days_to_delivery'] = (master.order_estimated_delivery_date - master.order_purchase_timestamp).dt.days

master['order_purchase_weekday'] = master['order_purchase_timestamp'].dt.day_name()

# Selection Indicator
master['S'] = (master['order_status'] != 'canceled').astype(int)


In [230]:
print(master.shape)
display(master.order_status.value_counts())
# Assumptions:
# As long as the status is not cancelled, we assume the customer completed the purchase

(118310, 43)


,count
order_status,
delivered,115723
shipped,1255
canceled,570
invoiced,376
processing,376
unavailable,7
approved,3


In [231]:
# Null Treatment
# Categorical
master.select_dtypes(include=['object']).fillna('Unknown', inplace=True)
# Numerical
master.fillna(0, inplace=True)

**Problem Set-Up**:


1.   Endogenous Variable: price is partially determined by demand. **Need Instrumental Variable.**

2.   Censored Data: decision to not purchase is not observable. **Need Selection Correction.**

Dynamic pricing requires a demand model/ demand curve. A demand curve is the relationship between the quantity demanded (or propensity to purchase) and the price among other features.


## Instrumental Variable
We want to estimate the response of market demand to exogenous changes in market prices. Quantity demanded depends on prices, but prices are not exogenously given since they are determined in part by market demand. The instrument for price is a variable that is correlated with price but does not directly effect quantity demanded, ideally a supply-inducing variable.

**IV ($Z$) Requirements**
1.   **Relevance**: $\text{Cov}(Z,\text{price}) \neq 0$
2.   **Exclusion/ Validity**: $Z \perp ϵ$ where $ϵ$ unobserved demand shock.

Some candidates:


*   Supply-side shock: supplier cost change, wholesale price, shipping cost (shift retail price but doesn't directly affect demand)
*   Day-of-week, temporary promotions (only if random scheduling)
* Regional difference: warehouse stock levels, local supply constraints (only if no difference in preference across regions)







In [232]:
# Possible Instrumental Variables
iv = [
    'order_purchase_weekday', # day of the week
    'freight_value', # shipping cost
    'seller_state', # proxy for regional supply shock
     ]

In [233]:
price = ['price']

covariates = [
       'days_to_delivery',
       'customer_zip_code_prefix',
      #  'customer_city',
       'customer_state',
       'product_category_name',
       'product_name_lenght',
       'product_description_lenght',
       'product_photos_qty',
       'product_weight_g',
       'product_length_cm',
       'product_height_cm',
       'product_width_cm',
       'seller_zip_code_prefix',
      #  'seller_city',
       'seller_state',
       'payment_sequential',
       'payment_type',
       'payment_installments',
       'payment_value',
       'review_score',
       ]

In [234]:
# Empirical Check for Relevance
X = master[covariates]
Z = master[iv]
y = master[price]

In [235]:
XZ = sm.add_constant(pd.concat([X, Z], axis=1))

In [236]:
# Empirical Check for Relevance
import statsmodels.api as sm

XZ = sm.add_constant(pd.concat([X, Z], axis=1))

# One-hot encode categorical columns in XZ
XZ_encoded = pd.get_dummies(XZ, columns= XZ.select_dtypes(include=['object']).columns,
                            dummy_na=False)

In [237]:
# An overview look of the IV. F Statistic is > 10, the instruments chosen are strong
relevance = sm.OLS(y, XZ_encoded.astype(float)).fit()
print(relevance.fvalue)

1372.4823046484248


In [194]:
# # Conditional Regressions (Sanderson-Windmeijer, 2016)
# # We have strong instruments
# X = master[covariates]
# Z = master[iv]
# y = master[price]

# Z_encoded = pd.get_dummies(Z,
#                             columns=Z.select_dtypes(include=['object']).columns,
#                             dummy_na=False)
# X_encoded = pd.get_dummies(X,
#                             columns=X.select_dtypes(include=['object']).columns,
#                             dummy_na=False)

# for z in Z_encoded.columns:
#     print(f"Conditional Regression for {z}:")
#     other_Z = Z_encoded.drop(columns=[z])
#     XZ_other = sm.add_constant(pd.concat([X_encoded, other_Z], axis=1))
#     # y = master[price]
#     y=Z_encoded[z]
#     model = sm.OLS(y, XZ_other.astype(float)).fit()

#     f_stat = model.fvalue
#     p_val = model.f_pvalue

#     print(f"Conditional F-test for {z}: F={f_stat:.2f}, p={p_val:.4f}")

## Heckman Correction

*   **Definition** (Selection Equation): The individual sampling probability of each observation
*   **Definition** (Outcome Equation): The conditional expectation of the dependent variable

**Two Stages of Heckman Correction**
1.   Estimate the probability of purchase. Here, we will use XGBoost to estimate
$$\hat{p}_i = P(S_i =1|z_i)$$
where $S_i=1$ means the purchase is observed and $z_i$ are instruments/ exclusion variables, affecting selection, but not directly purchase outcome.
2.   Correct for self-selection by incorporating a transformation of these predicted individual probabilities as an explanatory variable. This is done by computing the **inverse Mills ratio** for each observation $i$
$$\lambda_i = \frac{\phi(\Phi^{-1}(\hat{p}_i))}{\hat{p}_i}$$
and use the ratio as a control in outcome regression.


In [238]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import norm

In [239]:
X = master[covariates]
Z = master[iv]
y = master[price].values

In [248]:
# Selection Model
X_selection = master[covariates+iv]
y_selection = master['S']

# XGBoost classifier for selection
ml_selector = XGBClassifier(eval_metric='logloss',
                            )

# Perform one-hot encoding on X_selection
X_selection_encoded = pd.get_dummies(X_selection,
                                     columns=X_selection.select_dtypes(include='object').columns,
                                     dummy_na=False)

ml_selector.fit(np.array(X_selection_encoded), np.array(y_selection))
# Predict selection probability per observation
p_hat = ml_selector.predict_proba(np.array(X_selection_encoded))[:, 1]
# Compute IMR (inverse Mills ratio) for Heckman-style correction
imr = norm.pdf(norm.ppf(np.clip(p_hat, 1e-6, 1-1e-6))) / np.clip(p_hat, 1e-6, 1-1e-6)
master['imr'] = imr

In [281]:
# Outcome model
from doubleml import DoubleMLData, DoubleMLPLR

X_outcome = master[covariates+['imr', 'S', 'price']]

# One-hot encode categorical columns in XZ
X_outcome_encoded = pd.get_dummies(
                          X_outcome,
                          columns=X_outcome.select_dtypes(include=['object']).columns,
                          dummy_na=False)

dml_data = DoubleMLData(data=X_outcome_encoded[X_outcome_encoded.S==1],
                        y_col='S', # outcome
                        d_cols='price', # treatment
                        x_cols= X_outcome_encoded.drop(columns=['S', 'price']).columns.tolist(), # covariates
                        )

# Double ML
ml_l = XGBRegressor(n_estimators=100, max_depth=3)
ml_m = XGBRegressor(n_estimators=100, max_depth=3)

dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)
dml_plr.fit()

# Treatment effect estimate
print("Theta_hat:", dml_plr.coef)
print("Std. Error:", dml_plr.se)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Theta_hat: [0.]
Std. Error: [0.]


/usr/local/lib/python3.12/dist-packages/doubleml/utils/_sensitivity.py:6: RuntimeWarning: invalid value encountered in divide
  psi_max_bias = np.divide(np.add(np.multiply(sigma2, psi_nu2), np.multiply(nu2, psi_sigma2)), np.multiply(2.0, max_bias))
